In [1]:
import numpy as np
import pandas as pd
import datetime

sessionData = pd.DataFrame()
lastid = -1
lastClickData = pd.DataFrame()
PARTITION = 128
PATH = "parts\clicks_p"

sChunk = 10000 # default chunk size of searching.
def dateparse(dt_str):
    return datetime.datetime.strptime(dt_str, '%Y-%m-%dT%H:%M:%S.%fZ')

def getSessionData(sid):
    global PARTITION
    sres = pd.DataFrame()
    filep = PATH+str(sid%PARTITION)+".csv"
    s = pd.read_csv(filep,parse_dates = [1], date_parser = dateparse)
    sres=s.loc[s['session'] == sid]
    if sres.empty:
        print("Session not found!")
        #loadSpin=['|','/','-','\\']
        #print(loadSpin[int(index/2)%4]+" Searching partition "+str(index)+"...",end="\r")
    return sres

def parseSession(session,time,item_id):
    print("======== Session "+str(session)+" ========")
    print("# Product:",item_id)
    print("# Purchase time:",time)
    global lastid, lastClickData
    counter = 0
    clickTime = []
    tgap = datetime.datetime.now()
    if session != lastid:
        lastid = session
        temp = getSessionData(session)
        lastClickData=temp.loc[temp['item_id'] == item_id]
    for index, srow in lastClickData.iterrows():
        if srow['timestamp'] < time:
            counter+=1
        clickTime.append(srow['timestamp'])
    print("Clicks before buy: "+str(counter)+"  |  First click: "+str(clickTime[0])+"  |  Last click: "+str(clickTime[-1]))
    print("First click until buy: "+str((time - clickTime[0]).total_seconds())+
          " sec.  |  Last click until buy: "+str((time - clickTime[-1]).total_seconds())+" sec.",end="\n\n")

# iterate through all buy sessions.
def getPurchaseDataA(length=-1):
    loop = 0
    for index,row in buys.iterrows():
        if loop >= length and length != -1:
            break;
        parseSession(row['session'],row['timestamp'],row['item_id'])
        loop += 1
        
# get data by item_id
def getPurchaseDataI(itemSID,length=-1):
    loop = 0
    df = buys.loc[buys['item_id'] == itemSID]
    if df.empty:
        print("Find by Item ID: No Result!")
    else:
        for index,row in df.iterrows():
            if loop >= length and length != -1:
                break;
            parseSession(row['session'],row['timestamp'],row['item_id'])
            loop += 1

# get data by session ID
def getPurchaseDataS(sessionSID,length=-1):
    loop = 0
    df = buys.loc[buys['session'] == sessionSID]
    if df.empty:
        print("Find by Session: No Result!")
    else:
        for index,row in df.iterrows():
            if loop >= length and length != -1:
                break;
            parseSession(row['session'],row['timestamp'],row['item_id'])
            loop += 1
            
def extractSession(sessionSID,sess,isTest=False):
    df1=sess[['item_id']].groupby('item_id')
    res = []
    if not isTest:
        sessBuys = buys.loc[buys['session'] == sessionSID]
        doBuy = not sessBuys.empty
        for id, group in df1:
            clicks = group.count()[0]
            if doBuy:
                if sessBuys.loc[sessBuys['item_id'] == id].empty:
                    buyI=0
                else:
                    buyI=1
            else:
                buyI = 0
            print("[Train Data] Parsing Session: "+str(sessionSID),end='\r')
            dict = {'session':sessionSID,'item_id':id,'clicks':clicks,'buy':buyI}
            res.append(dict)
    else:
        for id, group in df1:
            clicks = group.count()[0]
            print("[Test Data] Parsing Session: "+str(sessionSID),end='\r')
            dict = {'session':sessionSID,'item_id':id,'clicks':clicks}
            res.append(dict)
    return res


In [5]:
# Read and sort buy session data
buys = pd.read_csv("yoochoose-data\yoochoose-buys.dat",parse_dates = [1], date_parser = dateparse)
buys=buys.sort_values(by='session')

In [14]:
# display functionalities
getPurchaseDataA(3)
getPurchaseDataI(214706441,3)
getPurchaseDataS(420374,3) 

======== Session 11 ========
# Product: 214821371
# Purchase time: 2014-04-03 11:04:18.097000
Clicks before buy: 3  |  First click: 2014-04-03 10:45:29.873000  |  Last click: 2014-04-03 10:46:57.355000
First click until buy: 1128.224 sec.  |  Last click until buy: 1040.742 sec.

======== Session 11 ========
# Product: 214821371
# Purchase time: 2014-04-03 11:04:11.417000
Clicks before buy: 3  |  First click: 2014-04-03 10:45:29.873000  |  Last click: 2014-04-03 10:46:57.355000
First click until buy: 1121.544 sec.  |  Last click until buy: 1034.062 sec.

======== Session 12 ========
# Product: 214717867
# Purchase time: 2014-04-02 10:42:17.227000
Clicks before buy: 2  |  First click: 2014-04-02 10:30:13.176000  |  Last click: 2014-04-02 10:33:12.621000
First click until buy: 724.051 sec.  |  Last click until buy: 544.606 sec.

======== Session 33 ========
# Product: 214706441
# Purchase time: 2014-04-06 18:24:51.426000
Clicks before buy: 4  |  First click: 2014-04-06 18:17:33.199000  | 

In [29]:
# Extract given session data.
resL=[]
for i in range(15,64):
    print("============================= PARTITION "+str(i)+" START =============================")
    filep = PATH+str(i)+".csv"
    dfp = pd.read_csv(filep,parse_dates = [1], date_parser = dateparse)
    sg = dfp.groupby('session')
    for id, group in sg:
        resL.extend(extractSession(id,group))
    print("============================= PARTITION "+str(i)+" DONE =============================")
df = pd.DataFrame(resL)    
df.info()

============================= PARTITION 15 START =============================
Session: 7345551

KeyboardInterrupt: 

In [27]:
df = df.sort_values(by='session',ascending=True)
df.head(100)
df.to_csv('session_buy_data_2_14.csv',index=False)

,session,item_id,clicks,buy
0,2,214551617,1,0
1,2,214662742,2,0
2,2,214757390,1,0
3,2,214757407,1,0
4,2,214825110,1,0
...,...,...,...,...
2289168,269,214820182,2,0
11,386,214517450,1,0
17,386,214839791,1,0
16,386,214714927,1,0


In [3]:
resT=[]
test = pd.read_csv("yoochoose-data/yoochoose-test.dat",parse_dates = [1], date_parser = dateparse,nrows=100000)
sg = test.groupby('session')
for id, group in sg:
    resT.extend(extractSession(id,group,True))
dfresT = pd.DataFrame(resT)    
dfresT.info()
dfresT.to_csv('test_data_100k.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75454 entries, 0 to 75453
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   session  75454 non-null  int64
 1   item_id  75454 non-null  int64
 2   clicks   75454 non-null  int64
dtypes: int64(3)
memory usage: 1.7 MB


In [ ]:
# ARCHIVE: old search method
    def getSessionData(sid,part_size = sChunk):
        #search for specific session ids.
        # part_size = 10000
        #sid = 16004 # session id to search.\n",
        #print(\"Search partition size: \"+str(part_size),end=\"\\r\")\n",
        sres = pd.DataFrame()
        s = pd.read_csv("yoochoose-data\yoochoose-clicks.dat",parse_dates = [1], date_parser = dateparse, chunksize=part_size)
        index = 0
        for chunk in s:
            sres=chunk.loc[chunk['session'] == sid]
            if not sres.empty:
                #print(\"\\nFound session id in partition \"+str(index))
                widedf = pd.read_csv("yoochoose-data\yoochoose-clicks.dat", skiprows=range(1,part_size*(index)-int(0.5*part_size)),nrows=part_size*2,
                                 parse_dates = [1], date_parser = dateparse)
                sres=widedf.loc[widedf['session'] == sid]
                break\n",
            else:\n",
                loadSpin=['|','/','-','\\\\']
                print(loadSpin[int(index/2)%4]+" Searching partition "+str(index)+"...",end="\r")
            index+=1
        return sres